In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder
from sklearn.preprocessing import LabelEncoder

# Force CPU usage for TensorFlow
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.get_logger().setLevel('ERROR')

# Load the compas dataset
dataset = pd.read_csv("compas-scores-two-years.csv")

encoders = {}
for col in dataset.columns:
    if dataset[col].dtype == 'object':
        encoder = LabelEncoder()
        dataset[col] = encoder.fit_transform(dataset[col])

X = dataset.drop(columns=["two_year_recid"])
y = dataset["two_year_recid"]

# dropping names and ids
X.drop(columns=["id", "name", "first", "last"], inplace=True)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Train a simple RandomForest model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Define a simple TensorFlow model wrapper for WIT
class SimpleModelWrapper(tf.Module):
    def __init__(self, sklearn_model):
        self.sklearn_model = sklearn_model

    def __call__(self, features):
        predictions = self.sklearn_model.predict_proba(features)
        return tf.convert_to_tensor(predictions)

wrapped_model = SimpleModelWrapper(model)

examples = [{
    col: X_test[col][i] for col in X_test.columns
} for i in X_test.index]

config_builder = WitConfigBuilder(examples).set_custom_predict_fn(wrapped_model)

widget = WitWidget(config_builder)

widget.render()

2024-11-05 23:41:08.097564: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730830268.110318   17852 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730830268.113905   17852 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 23:41:08.127513: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'uses_json_input': True, 'inference_addre…